<a href="https://colab.research.google.com/github/PatrickCong/KB/blob/master/ms3_patrick.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import pickle
import io
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn import datasets
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier  # Import KNN model
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, recall_score, make_scorer

from sklearn.pipeline import make_pipeline


def clean_and_encode(df):
    # further cleaning and label encoding
    df2 = df.copy()

    df2 = df2[df2['Incidence_Rate'].notna()]
    c = df2['country']
    c_enc = LabelEncoder().fit(c)
    df2['country_encoded'] = c_enc.transform(c)

    p = df2['province']
    p_enc = LabelEncoder().fit(p)
    df2['provice_encoded'] = p_enc.transform(p)

    # sex_encoding
    df2.loc[df2['sex'] == 'male', 'sex_encoding'] = 0
    df2.loc[df2['sex'] == 'female', 'sex_encoding'] = 1

    df2 = df2.drop(columns=['sex', 'province', 'country', 'latitude', 'longitude', 'date_confirmation',
                            'additional_information', 'source'])

    return df2


# def train_and_save_XGBoost(X_train, y_train, filename):
#     model = XGBClassifier(max_depth=8)
#     model.fit(X_train, y_train)
#
#     # save the model to disk
#     pickle.dump(model, open(filename, 'wb'))
#
#
# def train_and_save_KNN(X_train, X_test, y_train, y_test, filename):
#     k_value = knn_overfitting(X_train, X_test, y_train, y_test)
#     model = KNeighborsClassifier(n_neighbors=k_value)
#     model.fit(X_train, y_train)
#
#     # save the model to disk
#     pickle.dump(model, open(filename, 'wb'))


def train_and_save_Random_forest(X_train, y_train, filename):
    model = RandomForestClassifier()
    model.fit(X_train, y_train)

    # save the model to disk
    pickle.dump(model, open('rf_classifier.pkl', 'wb'))


def train_test_eval(X_train, X_test, y_train, y_test, filename):
    loaded_model = pickle.load(open(filename, 'rb'))

    y_pred = loaded_model.predict(X_test)
    print("Prediction accuracy on test data:", accuracy_score(y_test, y_pred), "\n")

    y_ = loaded_model.predict(X_train)
    print("Prediction accuracy on training data:", accuracy_score(y_train, y_), "\n")

    print("Confusion Matrix:", "\n", metrics.confusion_matrix(y_test, y_pred), "\n")

    print("Classification Report:", "\n", classification_report(y_test, y_pred), "\n")

    # print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred),"\n")

    # print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred),"\n")

    # print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)),"\n")




df = pd.read_csv(io.StringIO(uploaded['joined_dataset.csv'].decode('utf-8')))






data = clean_and_encode(df)

X = (data.drop(columns='outcome'))
y = data.outcome

    # split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
print('Splitting dataset done')

print("Random Forests", "\n")
    # train_test_eval(X_train.drop(['sex_encoding'], axis=1), X_test.drop(['sex_encoding'], axis=1), y_train, y_test,
    #                 rf_filename)
    # train_and_save_Random_forest(X_train.drop(['sex_encoding'],axis =1), y_train, rf_filename)
    # print('Build models Done')



scoring = {'accuracy': make_scorer(accuracy_score),
            'recall': make_scorer(recall_score,average = 'weighted'),
            'deceased_recall':make_scorer(recall_score,average = None, labels=["deceased"])}

    # Create a based model
rfc = RandomForestClassifier()
grid_params = {
    'n_estimators': [1, 100, 1000],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [1,5,10,20],
    'min_samples_leaf' : [1,3,6,10]

    }
CV_rfc = GridSearchCV(rfc, grid_params, verbose=1, cv=3, n_jobs= -1, scoring = scoring, refit = 'deceased_recall')
result = CV_rfc.fit(X_train.drop(['sex_encoding'],axis =1), y_train)






    # Overfitting check
    # knn_overfitting(X_train.drop(['sex_encoding'],axis =1), X_test.drop(['sex_encoding'],axis =1), y_train, y_test)





Splitting dataset done
Random Forests 

Fitting 3 folds for each of 144 candidates, totalling 432 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 12.0min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed: 71.7min
[Parallel(n_jobs=-1)]: Done 432 out of 432 | elapsed: 231.5min finished


In [24]:
result.best_params_

{'max_depth': 20,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'n_estimators': 1}

In [25]:
result.best_score_

0.045178691840863115

In [26]:
result.cv_results_

{'mean_fit_time': array([  0.97401269,   7.35326831,  67.13517189,   0.89835334,
          7.51741664,  67.4238507 ,   0.89754963,   7.53696823,
         67.18807038,   0.9167335 ,   7.49097164,  67.15479128,
          0.89693332,   7.55118513,  66.8664391 ,   0.89824947,
          7.51081848,  66.88344304,   0.90344206,   7.55065775,
         66.96482579,   0.89498806,   7.5288012 ,  67.10607592,
          0.89150715,   7.64673654,  66.85900489,   0.900165  ,
          7.47324872,  67.11191209,   0.90285921,   7.66979448,
         66.96416187,   0.89872964,   7.52045711,  66.8422691 ,
          0.95544489,  13.97869976, 133.41778286,   0.95252426,
         14.08495132, 132.48986371,   0.95279741,  14.01119598,
        133.6627721 ,   0.97284937,  14.27533849, 133.66698678,
          0.94947712,  13.99972852, 133.97608987,   0.94560806,
         14.16051865, 131.86035466,   0.97009818,  14.21989338,
        134.89814385,   0.95807505,  14.0704577 , 133.40240788,
          0.95322895,  

In [30]:
y_predict = CV_rfc.predict(X_test.drop(['sex_encoding'],axis =1))

In [31]:
print(y_predict)

['hospitalized' 'hospitalized' 'nonhospitalized' ... 'nonhospitalized'
 'hospitalized' 'nonhospitalized']


In [34]:
print("Prediction accuracy on test data:",accuracy_score(y_test, y_predict))

Prediction accuracy on test data: 0.8161500737226746
